In [39]:
import numpy as np 
import pandas as pd
from datetime import datetime

In [40]:
data_1hour = pd.read_csv('AAPL_full_1hour_adjsplit.csv')
data_1hour.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
data_1day = pd.read_csv('AAPL_full_1day_adjsplit.csv')
data_1day.columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
data_1hour_dropped = data_1hour.drop([0,1,2,3,4,5,6,7,8,9], axis =0)
data_1hour_dropped = pd.DataFrame(data_1hour_dropped)
data_1hour_dropped['Date'] = pd.to_datetime(data_1hour_dropped['Date'], format="%Y-%m-%d %H:%M:%S")
data_1hour_dropped["Hourly Log Return"] = np.log(data_1hour_dropped['Close']).diff()

In [41]:
#Creates a dataframe with columns "Date" and "Intraday Volatility"

def calculate_intraday_volatility(dataframe):

    dataframe['Date'] = pd.to_datetime(dataframe['Date'], format="%Y-%m-%d %H:%M:%S")
    date_log_return_df = dataframe[['Date', 'Hourly Log Return']]
    date_log_return_df.set_index('Date', inplace=True)
    intraday_volatility = date_log_return_df.resample('D').apply(lambda x: (x ** 2).sum(axis=0) / x.notna().sum(axis=0)).apply(lambda x: x ** 0.5)
    intraday_volatility.reset_index(inplace=True)
    intraday_volatility.columns = ['Date', 'Intraday_Volatility']

    return intraday_volatility

result_df = calculate_intraday_volatility(data_1hour_dropped)

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_64382/886415476.py:8: RuntimeWarning: invalid value encountered in double_scalars
  intraday_volatility = date_log_return_df.resample('D').apply(lambda x: (x ** 2).sum(axis=0) / x.notna().sum(axis=0)).apply(lambda x: x ** 0.5)


In [42]:
#Checks that the volatility has been calculated for each data (i.e. the volatility will be greater than zero )

def remove_zero_volatility(dataframe):

    filtered_dataframe = dataframe[dataframe['Intraday_Volatility'] != 0]
    return filtered_dataframe

newresults = remove_zero_volatility(result_df)


In [43]:
#Developing feature variables

data_1day['Date'] = pd.to_datetime(data_1day['Date'])
newresults['Date'] = pd.to_datetime(newresults['Date'])
data_1day.set_index('Date', inplace=True)
newresults.set_index('Date', inplace=True)
data_1day["Daily Volatility"] = newresults["Intraday_Volatility"]
data_1day['Range'] = data_1day['High'] - data_1day['Low']
data_1day['Return'] = np.log(data_1day['Close']).diff()
data_1day['Return_Squared'] = np.square(data_1day['Return'])
data_1day = data_1day.dropna()

In [44]:
pip install pandas_ta

Note: you may need to restart the kernel to use updated packages.


In [45]:
#Obtaining EMAF feature

import yfinance as yf
import pandas_ta as ta
data = yf.download(tickers = 'AAPL', start = '2005-01-05', end = '2024-01-20')
data['EMAF'] = ta.ema(data.Close, length = 20)
data_1day["EMAF"] = data["EMAF"]
data = data_1day.dropna()
daily_volatility = data["Daily Volatility"]

[*********************100%%**********************]  1 of 1 completed


In [46]:
#Making target variable
target = [1 if daily_volatility[i] > daily_volatility[i - 1] else -1 for i in range(1, len(daily_volatility))]
target = [0] + target
df = pd.DataFrame({'daily_volatility': daily_volatility})
df['target'] = np.where(df['daily_volatility'] > df['daily_volatility'].shift(1), 1, 0)
df['target'].fillna(0, inplace=True)
data['target'] = df['target']
data_full = data.dropna()

/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_64382/694950117.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  target = [1 if daily_volatility[i] > daily_volatility[i - 1] else -1 for i in range(1, len(daily_volatility))]
/var/folders/cb/9kkmszhd7fx9_93t2qb1_s640000gn/T/ipykernel_64382/694950117.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['target'] = df['target']


In [47]:
#Final dataset
data_full = data_full[["Return_Squared","Return", "Range", "Volume",  "EMAF",  "Daily Volatility", "target"]]

In [ ]:
data_full.to_csv(r"Intraday Volatility Dataset.csv")